## Check acquisition probabilities after disabling MS flag for acquisition

This notebook compares acquisition predictions following the operational disabling of the Multiple Stars Suspected (MS) flag.  It compares the MS-enabled model (before 2016:039) to the MS-disabled model (after 2016:039) and confirms that faint stars are indeed being acquired and identified by the OBC at the expected (much higher) rate.

In particular, as of 2016:103 (44 days after operational disable):

- Attempted acquisitions: 275
- Successful ACA acquisition and OBC identification: 251
- MS-enabled model prediction: 143.8
- MS-disabled model prediction: 233.5
 
The MS-disabled model is fully consistent with observation ($P \sim 0.12$) while the MS-enabled model is ruled out at very high confidence ($P << 10^{-8}$).

In [1]:
from __future__ import division

import sys
sys.path.insert(0, '/home/aldcroft/git/chandra_aca')

from astropy.time import Time
import matplotlib.pyplot as plt
import tables
import numpy as np
from astropy.table import Table
from chandra_aca import star_probs
from scipy.stats import poisson
%matplotlib inline

In [2]:
# Just before first obsid of FEB0816 when MS flag was disabled
# for acquisition.
tstart = Time('2016:039:09:23:10.489').cxcsec
filename = '/proj/sot/ska/data/acq_stats/acq_stats.h5'
if 'stats' not in globals():
    with tables.openFile(filename) as h5:
        stats = h5.root.data.readWhere('guide_tstart > {}'.format(tstart))
    stats = Table(stats)

    stats.sort('mag_aca')

In [3]:
def plot_n_exp_obs(stats, mag_limit_low=10.0, mag_limit_high=10.7, ms_enabled=True):
    ok = (stats['mag_aca'] > mag_limit_low) & (stats['mag_aca'] < mag_limit_high)
    stats = stats[ok]

    star_probs.set_fit_pars(ms_enabled)
    acq_probs = star_probs.acq_success_prob(date=stats['guide_tstart'],
                                            t_ccd=stats['ccd_temp'],
                                            mag=stats['mag_aca'],
                                            color=stats['color1'])

    n_acq = len(acq_probs)
    n_exp = np.sum(acq_probs)
    n_obs = np.sum(stats['acqid'])
    if len(acq_probs) > 30:
        print('Too many stars for computing cumulative probability (will take too long)')
        cum_n_acq_probs = None
    else:
        _, cum_n_acq_probs = star_probs.prob_n_acq(acq_probs)

    # cum_n_acq_probs[i] is probability of acquiring i or fewer
    # 1 - cum_n_acq_probs[i] is probability of acq'ing (i+1) or more
    # 1 - cum_n_acq_probs[i-1] is probability of acq'ing i or more
    print('N acq attempts = {}'.format(n_acq))
    print('Expected success = {:.2f}'.format(n_exp))
    print('Observed success = {}'.format(n_obs))
    dist = poisson(n_exp)
    print('Probability of {} or more given mean {} is {:.4g}'.format(n_obs, n_exp, 1 - dist.cdf(n_obs)))
    if cum_n_acq_probs is not None:
        n = np.arange(1, n_acq)
        p = 1 - cum_n_acq_probs[n - 1]
        p0 = 1 - cum_n_acq_probs[n_obs - 1]
        plt.plot(n, p)
        plt.grid()
        print('Probability {} or more successes = {:.3g}'
             .format(n_obs, p0))
        plt.plot([n_obs, n_obs], [0, 1], '--r')
        plt.plot([n_obs], [p0], 'or')
        plt.xlabel('N success')
        plt.ylabel('Probability')
        plt.title('Probability(N or more successes)');

### Cumulative probability using the old (MS-enabled) acq probability model

In [4]:
plot_n_exp_obs(stats, mag_limit_low=10.0, ms_enabled=True)

Too many stars for computing cumulative probability (will take too long)
N acq attempts = 275
Expected success = 143.77
Observed success = 251
Probability of 251 or more given mean 143.773874267 is 2.22e-16


In [5]:
plot_n_exp_obs(stats, mag_limit_low=10.3, ms_enabled=True)

Too many stars for computing cumulative probability (will take too long)
N acq attempts = 50
Expected success = 15.20
Observed success = 42
Probability of 42 or more given mean 15.2035131379 is 4.18e-09


### Cumulative probability using the new (MS-disabled) acq probability model

In [6]:
plot_n_exp_obs(stats, mag_limit_low=10.0, ms_enabled=False)

Too many stars for computing cumulative probability (will take too long)
N acq attempts = 275
Expected success = 233.50
Observed success = 251
Probability of 251 or more given mean 233.50262313 is 0.1202


In [7]:
plot_n_exp_obs(stats, mag_limit_low=10.3, ms_enabled=False)

Too many stars for computing cumulative probability (will take too long)
N acq attempts = 50
Expected success = 37.08
Observed success = 42
Probability of 42 or more given mean 37.0753718898 is 0.1848
